In [1]:
import os

In [2]:
%pwd

'h:\\Text Summarizer\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Text Summarizer\\Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name
        )

        return data_transformation_config 
    


In [8]:
from TextSummarizer.logging import logger
from transformers import  AutoTokenizer
from datasets import load_dataset, load_from_disk,Dataset
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

c:\ProgramData\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-04 18:18:30,669: INFO: config: PyTorch version 2.2.2 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['text'], max_length= 1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length= 128, truncation=True)
        
        return{
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        df = pd.read_csv(self.config.data_path)

        dataset_bbc = Dataset.from_pandas(df)
        
        # Split the dataset into train and test
        dataset_train_test = dataset_bbc.train_test_split(test_size=0.2)
        dataset_train = dataset_train_test['train']
        dataset_test = dataset_train_test['test']
        
        # Split the train dataset into train and validation
        dataset_train_val = dataset_train.train_test_split(test_size=0.1)
        dataset_train = dataset_train_val['train']
        dataset_val = dataset_train_val['test']
        
        # Now you can map the convert_examples_to_features function to each split
        dataset_train = dataset_train.map(self.convert_examples_to_features, batched=True)
        dataset_val = dataset_val.map(self.convert_examples_to_features, batched=True)
        dataset_test = dataset_test.map(self.convert_examples_to_features, batched=True)

        dataset_train.save_to_disk(os.path.join(self.config.root_dir, 'train_dataset'))
        dataset_val.save_to_disk(os.path.join(self.config.root_dir, 'val_dataset'))
        dataset_test.save_to_disk(os.path.join(self.config.root_dir, 'test_dataset'))

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-05-04 18:18:42,937: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-04 18:18:42,949: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-04 18:18:42,951: INFO: common: created directory at: artifacts]
[2024-05-04 18:18:42,952: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/1602 [00:00<?, ? examples/s]c:\ProgramData\anaconda3\envs\nlp\Lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 445/445 [00:00<00:00, 41402.48 examples/s]
